In [2]:
from personal.mysqldb import ToMysql
import pymysql
import pandas as pd
from dateutil.parser import parse
import numpy as np
import json
import decimal
import re
import string
from datetime import datetime
import time
import numba

np.set_printoptions(threshold=100000)
pd.options.display.max_rows = 100000
pd.options.display.max_columns = 800
pd.set_option('expand_frame_repr', False)


def get_good(x, y):
    z = ''
    if x >= 2:
        z = 'bad'
    elif (x < 2) & (y >= 6):
        z = 'good'
    else:
        z = 'unknow'
    return z

def get_mark(x,y):
    z=-99
    if x>=1:
        z=0
    elif (x<1) & (y>=6):
        z=1
    else:
        z=-99
    return z

def get_mark2(x):
    if x == "good":
        s = 1
    elif x=="bad":
        s = 0
    else:
        s = -99
    return s


@numba.jit()
def Chi2(df_arr, allBadRate):
    arr_except = df_arr[:, 1] * allBadRate
    arr = np.column_stack((df_arr[:, 2], arr_except))
    chi = (arr[:, 0] - arr[:, 1])**2/arr[:, 1]
    chi2 = sum(chi)
    return chi2


def ChiMerge_MaxInterval(df, col, target, max_interval=5):
    '''
        :param df: the dataframe containing splitted column, and target column with 1-0
        :param col: splitted column
        :param target: target column with 1-0
        :param max_interval: the maximum number of intervals. If the raw column has attributes less than this parameter, the function will not work
        :return: the combined bins
    '''

    col_levels = set(df[col])  # 将col列处理为集合，并排除掉缺失值
    col_levels.remove(-99) if -99 in col_levels else None
    col_levels = sorted(list(col_levels))
    col_original_count = len(col_levels)
    if col_original_count <= max_interval:
        print('The original levels for {} is less than or equal to max intervals'.format(col))
    else:
        # Step 1: group the dataset by col and work out the total count & bad count in each level of the raw column
        total = df.groupby([col])[target].count()
        df_total = pd.DataFrame({'total': total})
        bad = df.groupby([col])[target].sum()
        df_bad = pd.DataFrame({'bad': bad})
        regroup = df_total.merge(df_bad, left_index=True, right_index=True, how='left')

        regroup.reset_index(level=0, inplace=True)


        N = sum(regroup['total'])
        B = sum(regroup['bad'])

        allBadRate = B * 1.0/N   # 统计全部的坏样本率

        # initially, each single attribute forms a single interval

        group_intervals = [[i] for i in col_levels]

        group_num = len(group_intervals)
        while group_num > max_interval:
            # in each step of iteration, we calcualte the chi-square value of each atttribute
            chisq_list = []
            for i in group_intervals:
                df2 = regroup.loc[regroup[col].isin(i)]
                df_arr = df2.values
                chisq = Chi2(df_arr, allBadRate)

                chisq_list.append(chisq)

            # find the interval corresponding to minimum chi-square, and combine with the neighbore with smaller chi-square
            min_position = chisq_list.index(min(chisq_list))

            if min_position == 0:
                combined_position = 1
            elif min_position == group_num - 1:
                combined_position = min_position - 1
            else:  ## 如果在中间，则选择左右两边卡方值较小的与其结合
                if chisq_list[min_position - 1] < chisq_list[min_position + 1]:
                    combined_position = min_position - 1
                else:
                    combined_position = min_position + 1
            group_intervals[min_position] = group_intervals[min_position] + group_intervals[combined_position]

            # after combining two intervals, we need to remove one of them
            group_intervals.remove(group_intervals[combined_position])
            group_num = len(group_intervals)

        group_intervals = [sorted(i) for i in group_intervals]
        cut_off_points = [i[-1] for i in group_intervals[:-1]]

        return cut_off_points


def ChiMerge_MinChisq(df, col, target, confidenceVal=3.841):
    '''
    :param df: the dataframe containing splitted column, and target column with 1-0
    :param col: splitted column
    :param target: target column with 1-0
    :param confidenceVal: the specified chi-square thresold, by default the degree of freedom is 1 and using confidence level as 0.95
    :return: the splitted bins
    '''
    col_levels = set(df[col])
    total = df.groupby([col])[target].count()
    total = pd.DataFrame({'total': total})
    bad = df.groupby([col])[target].sum()
    bad = pd.DataFrame({'bad': bad})
    regroup = total.merge(bad, left_index=True, right_index=True, how='left')
    regroup.reset_index(level=0, inplace=True)
    N = sum(regroup['total'])
    B = sum(regroup['bad'])
    overallRate = B * 1.0 / N
    col_levels = sorted(list(col_levels))
    groupIntervals = [[i] for i in col_levels]
    groupNum = len(groupIntervals)
    while (1):  # the termination condition: all the attributes form a single interval; or all the chi-square is above the threshould
        if len(groupIntervals) == 1:
            break
        chisqList = []
        for interval in groupIntervals:
            df2 = regroup.loc[regroup[col].isin(interval)]
            chisq = Chi2(df2, 'total', 'bad', overallRate)
            chisqList.append(chisq)
        min_position = chisqList.index(min(chisqList))
        if min(chisqList) >= confidenceVal:
            break
        if min_position == 0:
            combinedPosition = 1
        elif min_position == groupNum - 1:
            combinedPosition = min_position - 1
        else:
            if chisqList[min_position - 1] <= chisqList[min_position + 1]:
                combinedPosition = min_position - 1
            else:
                combinedPosition = min_position + 1
        groupIntervals[min_position] = groupIntervals[min_position] + groupIntervals[combinedPosition]
        groupIntervals.remove(groupIntervals[combinedPosition])
        groupNum = len(groupIntervals)
    return groupIntervals


def BadRateEncoding(df, col, target):
    '''
    :param df: dataframe containing feature and target
    :param col: the feature that needs to be encoded with bad rate, usually categorical type
    :param target: good/bad indicator
    :return: the assigned bad rate to encode the categorical fature
    '''
    total = df.groupby([col])[target].count()
    total = pd.DataFrame({'total': total})
    bad = df.groupby([col])[target].sum()
    bad = pd.DataFrame({'bad': bad})
    regroup = total.merge(bad, left_index=True, right_index=True, how='left')
    regroup.reset_index(level=0, inplace=True)
    regroup['bad_rate'] = regroup.apply(lambda x: x.bad * 1.0 / x.total, axis=1)
    br_dict = regroup[[col, 'bad_rate']].set_index([col]).to_dict(orient='index')

    df['badRateEnconding'] = df.loc[:, ['data_id', col, target]].apply(lambda x: br_dict[x[col]]['bad_rate'], axis=1)
    return {'encoding': df, 'br_rate': br_dict}


## 给每个值重新编码为分段标签——对应ChiMerge_MinChisq方法
def get_cat(x, tt):
    z = ''
    for i in tt:
        print(i)
        if min(i) <= x <= max(i):
            z = str(round(min(i), 2)) + '~' + str(round(max(i), 2))
    return z

## 给每个值重新编码为分段标签——对应ChiMerge_MaxInterval方法
def get_point_dur(x, point):
    z = ''
    if point:
        num_point = len(point) + 1
        if (x != -99):
            if x <= point[0]:
                z = '0- ~' + str(point[0])
            elif x > point[-1]:
                z = '{}- '.format(num_point-1) + str(point[-1]) + '~'
            else:
                for i in range(0, num_point-1):
                    if point[i] < x <= point[i+1]:
                        z = '{}- '.format(i+1) + str(point[i]) + '~' + str(point[i + 1])
        else:
            z = '-99'
    return z


def chi_trans(df, ChiMerge_list, target, onkey, save_path):
    df_chis = df.loc[:, ['data_id', 'user_mark']]  # 切片，先填充缺失值为-99
    code_list = []
    for col in ChiMerge_list:
        print(col)
        df[col] = df[col].replace('-', None).astype(float)

        df[col] = df[col].map(lambda x: round(x / 1000, 1)) if abs(df[col].mean()) >= 1000 else df[col]
        df_chi = df.loc[:, [onkey, col, target]]
        df_chi = df_chi.fillna(-99)
        split_box = ChiMerge_MaxInterval(df_chi, col, target, max_interval=6)
        print(split_box)
        code_dict = {'col_name': col, 'split_box': split_box}
        if split_box:
            df_chi[col] = df_chi[col].apply(lambda x: get_point_dur(x, split_box))
        df_chi = df_chi.loc[:, [onkey, col]]
        df_chis = pd.merge(df_chis, df_chi, how='left', on=onkey)

        pd.DataFrame.to_csv(df_chis, save_path, sep=',')
        code_list.append(code_dict)
        j = json.dumps(code_list)
        with open(r'E:\hoomsun_data\analysis\models\split_code' + save_path[len(save_path)-10: len(save_path)-4:] + '.json', 'w') as j_file:
            j_file.write(j)
        j_file.close()
        print(datetime.now())

    return df_chis


# # 计算WOE与IV
def CalcWOE(df, col, target, excel_path):
    '''
    :param df: dataframe containing feature and target
    :param col: 注意col这列已经经过分箱了，现在计算每箱的WOE和总的IV。
    :param target: good/bad indicator
    :return: 返回每箱的WOE(字典类型）和总的IV之和。
    '''
    total = df.groupby([col])[target].count()
    total = pd.DataFrame({'total': total})
    bad = df.groupby([col])[target].sum()
    bad = pd.DataFrame({'bad': bad})
    regroup = total.merge(bad, left_index=True, right_index=True, how='left')
    regroup.reset_index(level=0, inplace=True)
    N = sum(regroup['total'])
    B = sum(regroup['bad'])
    regroup['good'] = regroup['total'] - regroup['bad']
    G = N - B
    regroup['bad_pcnt'] = regroup['bad'].map(lambda x: x*1.0/B)
    regroup['good_pcnt'] = regroup['good'].map(lambda x: x * 1.0 / G)
    regroup['WOE'] = regroup.apply(lambda x: np.log(x.good_pcnt*1.0/x.bad_pcnt), axis = 1)
    WOE_dict = regroup[[col, 'WOE']].set_index(col).to_dict(orient='index')
    regroup['IV'] = regroup.apply(lambda x: (x.good_pcnt-x.bad_pcnt)*np.log(x.good_pcnt*1.0/x.bad_pcnt), axis =1)
    IV = sum(regroup['IV'])
    pd.DataFrame.to_excel(regroup, excel_writer=excel_path, sheet_name=col)
    return {"WOE": WOE_dict, 'IV': IV}

# # badRate单调检查  检查分箱以后每箱的bad_rate的单调性，如果不满足，那么继续进行相邻的两箱合并，知道bad_rate单调为止。(可以放宽到U型)
def BadRateMonotone(df, sortByVar, target):
    # df[sortByVar]这列数据已经经过分箱
    df = df[df[sortByVar] != -99]
    df2 = df.sort([sortByVar])
    total = df2.groupby([sortByVar])[target].count()
    total = pd.DataFrame({'total': total})
    bad = df2.groupby([sortByVar])[target].sum()
    bad = pd.DataFrame({'bad': bad})
    regroup = total.merge(bad, left_index=True, right_index=True, how='left')
    regroup.reset_index(level=0, inplace=True)
    combined = zip(regroup['total'], regroup['bad'])
    badRate = [x[1]*1.0/x[0] for x in combined]
    badRateMonotone = [badRate[i]<badRate[i+1] for i in range(len(badRate)-1)]
    Monotone = len(set(badRateMonotone))
    if Monotone == 1:
        return True
    else:
        return False




In [3]:
read_path = r"d:\hoomsun_data\analysis\models\model_data.xlsx"
df = pd.read_excel(read_path)
print(df.shape)
df = df[df.credit_label == 1]
print(df.shape)
# df['user_mark'] = df.apply(lambda x: get_mark(x.history_max_qici, x.current_period), axis=1)
df['user_mark'] = df['classification'].map(get_mark2)
df = df[df.user_mark != -99]

print(df.shape)
df.head()

# 手工调整后重新分箱


(58277, 393)
(39156, 393)
(31363, 394)


,loan_id,apply_amount,apply_period,product_name,product_rate,cust_type,cust_sex,marital_status,rresidence_prov_name,house_prov_name,live_conditions,company_prov_name,company_kind_va,private_type,industry,premises_val,position_val,property_type,card_account,card_notsettled,card_overdue,card_90overdue,card_guaranty,housing_loan_account,housing_loan_notsettled,housing_loan_overdue,housing_loan_90overdue,housing_loan_guaranty,other_loan_account,other_loan_notsettled,other_loan_overdue,other_loan_90overdue,other_loan_guaranty,card_count_only_rmb,card_count_card_notsettled,card_date_first_status,card_credit_history,card_credit_history_normal,card_award_first,card_award_first_normal,card_award_new,card_award_highest,card_award_highest_goodbank,card_award_used_highest,card_award_ratio_highest,card_award_sum,card_award_used,card_award_used_ratio,card_exceeding_count_,card_exceeding_count_ratio,card_exceeding_money_sum,card_exceeding_money_max,card_overdue_count,card_overdue_money_sum,card_overdue_money_max,card_overdue_months_in_5y,card_overdue_months_in_2y,card_overdue_months_in_1y,card_overdue_over90_in_5y,card_overdue_over90_in_2y,card_overdue_over90_in_1y,card_overdue_highest,card_overdue_ratio,card_overdue_0_10,card_overdue_10,card_overdue_20,card_overdue_30,card_overdue_10_extend_1y,card_overdue_20_extend_1y,card_overdue_30_extend_1y,card_overdue_10_extend_2y,card_overdue_20_extend_2y,card_overdue_30_extend_2y,card_decrease,card_freeze,card_doubtful,card_unactivated,loan_total_account,loan_notsettled_account,loan_settle_account,loan_settle_past6m,loan_settle_past3m,loan_settle_past1m,loan_settle_past6m_ratio,loan_settle_past3m_ratio,loan_settle_past1m_ratio,loan_settle_next1m,loan_settle_next3m,loan_settle_next6m,loan_settle_next12m,loan_award_past6m,loan_award_past3m,loan_award_past1m,loan_award_next1m,loan_award_next3m,loan_award_next6m,loan_award_next12m,loan_moneymonthly_next3m,loan_moneymonthly_next6m,loan_moneymonthly_next12m,loan_count_bank,loan_ratio_bank,loan_count_bank_country_stock,loan_ratio_bank_country_stock,loan_count_bank_commerce,loan_ratio_bank_commerce,loan_count_bank_village,loan_ratio_bank_village,loan_count_bank_rcu_commerce,loan_ratio_bank_rcu_commerce,loan_count_bank_foreign,loan_ratio_bank_foreign,loan_count_mcc,loan_ratio_mcc,loan_count_cfc,loan_ratio_cfc,loan_count_trust,loan_ratio_trust,loan_count_afc,loan_ratio_afc,loan_count_weixin,loan_ratio_weixin,loan_count_mayi,loan_ratio_mayi,loan_history_weixin,loan_award_weixin_last,loan_history_mayi,loan_award_mayi_last,loan_count_check,loan_ratio_check,loan_count_house,loan_ratio_house,loan_award_house,loan_award_house_avg,loan_count_afl,loan_ratio_afl,loan_award_afl,loan_award_afl_avg,loan_count_consume,loan_ratio_consume,loan_award_consume,loan_award_consume_avg,loan_count_operate,loan_ratio_operate,loan_award_operate,loan_award_operate_avg,loan_count_peasant,loan_ratio_peasant,loan_award_peasant,loan_award_peasant_avg,loan_count_car,loan_ratio_car,loan_award_car,loan_award_car_avg,loan_overdue_month_5y,loan_overdue_month_2y,loan_overdue_month_1y,loan_90overdue_5y,loan_90overdue_2y,loan_90overdue_1y,loan_house_overdue_month_5y,loan_house_overdue_month_2y,loan_house_overdue_month_1y,loan_house_90overdue_5y,loan_house_90overdue_2y,loan_house_90overdue_1y,loan_overdue_highest,loan_overdue_house,loan_overdue_ratio_10,loan_overdue_ratio_20,loan_overdue_ratio_30,loan_overdue_10_extend_1y,loan_overdue_20_extend_1y,loan_overdue_30_extend_1y,loan_overdue_10_extend_2y,loan_overdue_20_extend_2y,loan_overdue_30_extend_2y,loan_count_normal,loan_ratio_normal,loan_count_settle,loan_ratio_settle,loan_count_overdue,loan_ratio_overdue,loan_count_doubtful,loan_count_out,loan_house_repaymonth,loan_count_normal_in_1y,loan_ratio_normal_in_1y,loan_count_settle_in_1y,loan_ratio_settle_in_1y,loan_count_overdue_in_1y,loan_ratio_overdue_in_1y,loan_count_doubtful_in_1y,loan_count_out_in_1y,loan_house_moneymonthly,loan_other_moneymonthly,loan_moneymonthly,loan_moneymonthly_notmortgage

In [ ]:
def re_trans(df, ChiMerge_list, target, onkey, save_path):
    df_chis = df.loc[:, ['loan_id', 'user_mark', 'classification', 'con_product_name']]
    for var_li in lisan_list:
        df_li = df.loc[:, ['loan_id', var_li]]
        df_chis = pd.merge(df_chis, df_li, how='left', on='loan_id')
    for col in ChiMerge_list:
        print(col)
        df[col] = df[col].replace('-', None).astype(float)
        df_chi = df.loc[:, [onkey, col, target]]
        df_chi = df_chi.fillna(-99)
        with open(r'd:\hoomsun_data\analysis\models\split_code_fix.json') as j_file:
            j = json.load(j_file)
        for i in j:
            if i['col_name'] == col:
                split_box = i['split_box']
                if split_box:
                    df_chi[col] = df_chi[col].apply(lambda x: get_point_dur(x, split_box))
            df_chi = df_chi.loc[:, [onkey, col]]

        df_chis = pd.merge(df_chis, df_chi, how='left', on=onkey)
        pd.DataFrame.to_csv(df_chis, save_path, sep=',', encoding="utf_8_sig")
        print(datetime.now())

    return df_chis

df_list = pd.read_excel(r'd:\hoomsun_data\analysis\models\建模字典v4(2).xlsx')
lisan_list = list(df_list[(df_list.属性 == 'cat') & (df_list['是否使用']==1)]['英文'])
ChiMerge_list_recode = list(df_list[(df_list['属性']== 'num') & (df_list['是否使用']==1)]['英文'])
re_trans(df, ChiMerge_list_recode, 'user_mark', 'loan_id', save_path=r'd:\hoomsun_data\analysis\models\data_8.28.csv')


C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1472: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


card_account
2018-08-28 15:18:57.815500
card_notsettled
2018-08-28 15:18:59.116500
card_overdue
2018-08-28 15:19:00.468500
card_90overdue
2018-08-28 15:19:01.724500
card_guaranty
2018-08-28 15:19:02.991500
housing_loan_account
2018-08-28 15:19:04.273500
housing_loan_notsettled
2018-08-28 15:19:05.561500
housing_loan_overdue
2018-08-28 15:19:06.858500
housing_loan_90overdue
2018-08-28 15:19:08.150500
housing_loan_guaranty
2018-08-28 15:19:09.462500
other_loan_account
2018-08-28 15:19:10.828500
other_loan_notsettled
2018-08-28 15:19:12.205500
other_loan_overdue
2018-08-28 15:19:13.559500
other_loan_90overdue
2018-08-28 15:19:14.900500
other_loan_guaranty
2018-08-28 15:19:16.259500
card_count_only_rmb
2018-08-28 15:19:17.740500
card_count_card_notsettled
2018-08-28 15:19:19.274500
card_credit_history
2018-08-28 15:19:20.804500
card_credit_history_normal
2018-08-28 15:19:22.638500
card_award_first
2018-08-28 15:19:24.238500
card_award_first_normal
2018-08-28 15:19:26.179500
card_award_new


2018-08-28 15:25:10.274500
loan_count_out_in_1y
2018-08-28 15:25:13.143500
loan_house_moneymonthly
2018-08-28 15:25:16.327500
loan_other_moneymonthly
2018-08-28 15:25:19.477500
loan_moneymonthly
2018-08-28 15:25:22.699500
loan_moneymonthly_notmortgage
2018-08-28 15:25:25.979500
loan_award_total
2018-08-28 15:25:29.465500
loan_balance
2018-08-28 15:25:32.796500
loan_overdue_account
2018-08-28 15:25:35.881500
loan_award_overdue
2018-08-28 15:25:39.024500
loan_overdue_money
2018-08-28 15:25:42.146500
loan_money_1
2018-08-28 15:25:45.282500
loan_money_1_2
2018-08-28 15:25:48.471500
loan_money_2_10
2018-08-28 15:25:51.769500
loan_money_10_20
2018-08-28 15:25:54.936500
loan_money_20_50
2018-08-28 15:25:58.128500
loan_money_50_100
2018-08-28 15:26:01.444500
loan_money_100_200
2018-08-28 15:26:04.699500
loan_money_200
2018-08-28 15:26:07.911500
loan_money_over_20
2018-08-28 15:26:11.060500
loan_money_over_50
2018-08-28 15:26:14.194500
credit_1m
2018-08-28 15:26:17.437500
credit_2m
2018-08-28 1